Fetch info

In [ ]:
#%% function - deps - fetch
import sys
import os

# current_dir = os.path.dirname(__file__) # for py file
current_dir = os.getcwd() # for ipynb file
dep_dir = os.path.join(current_dir, 'deps')
sys.path.append(dep_dir)

from help_func_fetch import Classyfire_Fetcher, PubChem_Fetcher, _get_list_first

In [ ]:
#%% load data
import pandas as pd

raw_df = pd.read_csv('data.csv',header=None,names=['inchikey','smiles'])

In [ ]:
#%% fetch - pubchem

# may try several time to fullly cache data
# fetch pubchem basic info
pbc_f = PubChem_Fetcher(cache_folder = 'refinfo/pubchem', cache_file = 'refinfo/pubchem/pbc_cache.feather')
inchikey_list = raw_df['inchikey'].values.tolist()
results_basic = pbc_f.fetch_pubchem_basic(inchikey_list, mode = 'inchikey', use_service=True)

# fetch pubchem json through REST PUG
cid_list = results_basic['cid'].values.tolist()
results_json = pbc_f.fetch_pubchem_json(cid_list, mode = 'cid')


In [ ]:
#%% fetch - classyfire
from help_func_fetch import Classyfire_Fetcher

cly_f = Classyfire_Fetcher(cache_file = 'refinfo/classyfire/clf_cache.feather')
smi_list = raw_df['smiles'].tolist()
results_clf = cly_f.fetch_classfire(smi_list, mode = 'smiles')

In [ ]:
#%% Merge info
full_df = pd.merge(raw_df, results_basic[['inchikey', 'cid','casrn', 'dtxsid','iupac_name']], on='inchikey', how='left')
full_df = pd.merge(full_df, results_clf[['input', 'kingdom', 'superclass','class', 'subclass','direct_parent',]], left_on='smiles', right_on='input', how='left')
full_df = full_df.drop(columns = ['input'])
full_df = pd.merge(full_df, results_json.drop(columns = ['cid']), left_on = 'cid', right_on='input', how='left')
full_df = full_df.drop(columns = ['input'])
print(full_df.head(5))

full_df.to_feather('data_full.feather')

Calc mol similarity

In [ ]:
#%% function - deps - mol simi
from rdkit import Chem
from rdkit.Chem import AllChem
import numpy as np

import sys
import os

# current_dir = os.path.dirname(__file__) # for py file
current_dir = os.getcwd() # for ipynb file
dep_dir = os.path.join(current_dir, 'deps')
sys.path.append(dep_dir)

from help_func_mol import mol_simi_df

In [ ]:
#%% calc - pairwise, so take long time if too much rows

# import pandas as pd
# full_df = pd.read_feather('data_full.feather')

full_df['mol'] = full_df['smiles'].map(Chem.MolFromSmiles)
full_df = full_df[full_df['mol'].notna()].reset_index(drop=True)

results_simi = mol_simi_df(full_df, 'mol', n_jobs=-1)
# with open('simi.pkl', 'wb') as f:
#     pickle.dump(results_simi, f)

# with open('simi.pkl', 'rb') as f:
#     results_simi = pickle.load(f)

simi_weights = [1/3,1/3,1/3]
for dict1 in results_simi:
    main_simi, core_simi , msc_simi = dict1['simi']
    dict1['simi'] = sum([main_simi*simi_weights[0], core_simi*simi_weights[1], msc_simi*simi_weights[2]])


Define food related or synthetic

In [ ]:
#%% load data
import pandas as pd

full_df = pd.read_feather('data_full.feather')

In [ ]:
#%% initial foodrelated column

full_df['foodrelated'] = None

synthetic = full_df[['Industry_Uses', 'Consumer_Uses', 'Manufacturing']].notnull().any(axis=1)
food_relate = full_df['Associated_Foods'].notnull()  
food_safe = full_df['Food_Additive_ADI'].str.contains('No safety concern') & full_df['Food_Additive_ADI'].notnull()

full_df.loc[food_relate & ~synthetic, 'foodrelated'] = 'onlyfoodrelate'  
full_df.loc[food_relate & synthetic, 'foodrelated'] = 'mixinfo'  
full_df.loc[synthetic & ~food_relate, 'foodrelated'] = 'onlysynthetic'
full_df.loc[~food_relate & ~synthetic, 'foodrelated'] = 'unknown'  
full_df.loc[food_safe, 'foodrelated'] = 'onlyfoodrelate' 

full_df.drop(['mol'], axis=1).to_feather('data_full_before.feather')

In [ ]:
#%% info share - foodrelated column fill
from tqdm import tqdm

similarity_threshold = 0.7

results_simi1 = [sim_dict for sim_dict in results_simi if sim_dict['simi'] > similarity_threshold]
results_simi1 = pd.DataFrame(results_simi1)

modification_log = []

n = 0
max_run = 40

while True:
    n += 1
    unknown_len = len(full_df[full_df['foodrelated'] == 'unknown'])
    change_require = []

    for unknown_index in tqdm(full_df[full_df['foodrelated'] == 'unknown'].index, desc = f'Run {n}'):   
        similar_indexes = results_simi1[  
            (results_simi1['index1'] == unknown_index) | (results_simi1['index2'] == unknown_index)
        ]

        if len(similar_indexes) > 0:
            similar_rows = []

            for _ , row in similar_indexes.iterrows():
                if row['index1'] == unknown_index:
                    index  = row['index2']
                else:
                    index = row['index1']
                    
                if full_df.loc[index, 'foodrelated'] != 'unknown':
                    similar_rows.append({'index':index, 'simi':row['simi']})

            similar_rows = sorted(similar_rows, key=lambda x: x['simi'], reverse=True)
            similar_rows = similar_rows[:15]
            
            if len(similar_rows) > 0:

                full_df_simi_index = [row['index'] for row in similar_rows]
                full_df_simi_simi = [row['simi'] for row in similar_rows]
                similar_attributes = full_df.loc[full_df_simi_index, 'foodrelated'].tolist() 
                if similar_attributes.count('onlyfoodrelate') > 0:
                    if similar_attributes.count('mixinfo') == 0 and similar_attributes.count('onlysynthetic') == 0:  
                        old_value = full_df.at[unknown_index, 'foodrelated']
                        change_require.append(unknown_index)
                        
                        modification_log.append({'run':n, 'changed_index':unknown_index, 'changed_due_to_index':full_df_simi_index, 'changed_due_to_simi':full_df_simi_simi, 'old_value': old_value, 'new_value': 'onlyfoodrelate'})
        
    # print(len(change_require))
    full_df.loc[change_require, 'foodrelated'] = 'onlyfoodrelate'

    if unknown_len == len(full_df[full_df['foodrelated'] == 'unknown']) and n <= max_run:
        print(f'Total runs: {n}')
        break

full_df.drop(['mol'], axis=1).to_feather(f'data_full_after.feather')


In [ ]:
#%% visualize foodrelated column 

import matplotlib.pyplot as plt
from tqdm import tqdm

import sys
import os

# current_dir = os.path.dirname(__file__) # for py file
current_dir = os.getcwd() # for ipynb file
dep_dir = os.path.join(current_dir, 'deps')
sys.path.append(dep_dir)

from help_func_mol import add_stroke, get_mol_image
from help_func_network import auto_wrap_text

before_df = pd.read_feather('data_full_before.feather')
diff = pd.merge(before_df, full_df['foodrelated'], left_index = True, right_index=True, suffixes=('_old', '_new'), how='outer')  
different_rows = diff[diff['foodrelated_old'] != diff['foodrelated_new']]  
diff = different_rows[before_df.columns.tolist()[:-1] + ['foodrelated_old', 'foodrelated_new']].reset_index(drop=True)

n = 8
num_rows = (len(diff) + n - 1) // n
 
fig, axes = plt.subplots(nrows=num_rows, ncols=n, figsize=(2 * n, 2 * num_rows),dpi = 300)  

max_width = 0
max_height = 0
scale = 2

for idx, row in diff.iterrows():  
    row_num = idx // n
    col_num = idx % n

    name = row['iupac_name']
    cid = row['cid']
    if (cid != cid) or (cid == None):
        cid = None
    else:
        cid = str(int(cid))
    inchikey = row['inchikey']
    label = [name, f'cid: {cid}', inchikey]
    label = [i for i in label if i not in [None,np.nan]]
    label = '\n'.join(label)

    mol = Chem.MolFromSmiles(row['smiles'])  
    if mol is not None:  
        img = np.array(get_mol_image(mol,False,False,False))
        img_width, img_height = img.shape[1], img.shape[0]
        scaled_width = img_width * scale
        scaled_height = img_height * scale
        axes[row_num, col_num].imshow(img, extent=(0 - scaled_width / 2, 0 + scaled_width / 2, 
                                0 - scaled_height / 2, 0 + scaled_height / 2))
        
        max_width = max(max_width, scaled_width)
        max_height = max(max_height, scaled_height)

    group_info = f"{label}\n{row['foodrelated_old']} -> {row['foodrelated_new']}"
    group_info = auto_wrap_text(group_info) 

    axes[row_num, col_num].text(0.5, -0.1, group_info, fontsize=4, 
                                ha='center', va='top', 
                                transform=axes[row_num, col_num].transAxes
                                )    
for ax in axes.flatten():
    ax.set_xlim(-max_width/2, max_width/2)
    ax.set_ylim(-max_height/2, max_height/2)
    ax.axis('off')

for i in range(len(diff), num_rows * n):  
    axes[i // n, i % n].axis('off') 

plt.subplots_adjust(wspace=0, hspace=0) 
plt.tight_layout()  
fig.savefig('chemchangetofoodrelate.png',bbox_inches='tight', dpi=200, transparent=False) 
plt.show() 
# plt.close()

Ready network

In [ ]:
#%% function - deps - network plot 
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil
import os
import pandas as pd
from rdkit import Chem
import networkx as nx
import math

import sys
import os

# current_dir = os.path.dirname(__file__) # for py file
current_dir = os.getcwd() # for ipynb file
dep_dir = os.path.join(current_dir, 'deps')
sys.path.append(dep_dir)

from help_func_mol import add_stroke, get_mol_image
from help_func_network import interpolate_color, auto_wrap_text
from help_func_network import plot_graph, trim_edge, cal_layout, compute_grid_layout


In [ ]:
#%% load data

full_df = pd.read_feather(f'data_full_after.feather')
full_df['mol'] = full_df['smiles'].map(Chem.MolFromSmiles)
full_df = full_df[full_df['mol'].notna()].reset_index(drop=True)

full_df['casrn'] = full_df['casrn'].apply(_get_list_first)
full_df['dtxsid'] = full_df['dtxsid'].apply(_get_list_first)

import pickle
with open('simi.pkl', 'rb') as f:
    results_simi = pickle.load(f)

simi_weights = [1/3,1/3,1/3]
for dict1 in results_simi:
    main_simi, core_simi , msc_simi = dict1['simi']
    dict1['simi'] = sum([main_simi*simi_weights[0], core_simi*simi_weights[1], msc_simi*simi_weights[2]])


In [ ]:
#%% Build main networkx graph

df = full_df[['iupac_name','smiles','cid','inchikey','mol','superclass', 'class', 'subclass', 'direct_parent','casrn','foodrelated']]

G = nx.Graph()

for idx, row in tqdm(df.iterrows(), total=len(df), desc="Adding node to network",leave = False):
    
    smiles = row['smiles'] if row['smiles'] is not None else ''
    name = row['iupac_name'] if row['iupac_name'] is not None else ''
    inchikey = row['inchikey'] if row['inchikey'] is not None else ''
    foodrelated = row['foodrelated']

    cid = row['cid']
    if (cid != cid) or (cid == None):
        cid = None
    else:
        cid = f'cid: {str(int(cid))}'

    cas = row['casrn']
    if (cas != cas) or (cas == None):
        cas = None
    else:
        cas = f'cas: {str(cas)}'

    label = [cas, cid, inchikey]
    label = [i for i in label if i not in [None,np.nan]]
    label = '\n'.join(label) if len(label) > 0 else ''

    G.add_node(str(idx), name=name, smiles = smiles, inchikey=inchikey, label = label, foodrelated = foodrelated)
    
G.clear_edges()

simi_threshold = 0.7

for item in tqdm(results_simi, desc='Adding edges', leave=False):
    i = item['index1']
    j = item['index2']
    simi = item['simi']
    fraction = (simi - simi_threshold) / (1 - simi_threshold)

    if simi <= simi_threshold:
        continue
    
    G.add_edge(str(i), str(j), simi=simi)

folder_path = 'network'

os.makedirs(folder_path, exist_ok=True)

nx.write_graphml(G, f"{folder_path}/network1.graphml")

In [ ]:
#%% one-time, generate node community list

import json
import random
random.seed(42)

import numpy as np
np.random.seed(42)

from help_func_network import split_by_community, count_bottom_elements, deterministic_subgraph, trim_edge, convert_sets_to_lists
from networkx.algorithms import community

clusters = list(nx.connected_components(G))
clusters = sorted(clusters, key=lambda c: count_bottom_elements(c), reverse=True)

node_list = []

for c in clusters:
    if len(c)>=200:
        subG = deterministic_subgraph(G, c)
        subG = trim_edge(subG, keep_top_n = True, keep_span_tree = True, simi_threshold=0.7)
        
        temp_c = split_by_community(subG, sort = True, resolution = 0.3)
        node_list.extend(temp_c)

        print(f'before: {len(c)}')
        len_str = ','.join([str(len(sub)) for sub in temp_c])
        print(f'after: {len_str}')
    else:
        node_list.append(c)

node_list = convert_sets_to_lists(node_list)
node_list = sorted(node_list, key=lambda c: count_bottom_elements(c), reverse=True)

folder_path = 'network'

os.makedirs(folder_path, exist_ok=True)

with open(f'{folder_path}/community_group1.json', 'w') as f:
    json.dump(node_list, f)

print(len(node_list))

In [ ]:
#%% one-time, generate pos_list

import json

folder_path = 'network'

with open(f'{folder_path}/community_group1.json', 'r') as f:
    node_list = json.load(f)

from help_func_network import deterministic_subgraph, trim_edge, cal_layout, count_bottom_elements

folder_path = 'network'
pos_folder = 'pos'

if not os.path.exists(f'{folder_path}/{pos_folder}'):
    os.makedirs(f'{folder_path}/{pos_folder}')

# pos_list = []
for i, c in enumerate(tqdm(node_list)):
    subG = deterministic_subgraph(G, c)
    subG = trim_edge(subG, keep_top_n = True, keep_span_tree = True, simi_threshold=0.7)
    pos = cal_layout(subG)
    # pos_list.append(pos)
    json.dump(pos, open(f'{folder_path}/{pos_folder}/node_pos{i}.json', 'w'))
    
file_list = os.listdir(f'{folder_path}/{pos_folder}')
pos_list = [json.load(open(f'{folder_path}/{pos_folder}/{file}', 'r')) for file in file_list]

pos_list = sorted(pos_list, key=lambda c: len(list(c.keys())), reverse=True)
json.dump(pos_list, open(f'{folder_path}/node_pos1.json', 'w'))

In [ ]:
#%% trim edge for the main graph

import sys
import os

current_dir = os.getcwd() # for ipynb file
dep_dir = os.path.join(current_dir, 'deps')
sys.path.append(dep_dir)

from help_func_network import deterministic_subgraph, trim_edge, convert_sets_to_lists

edge_list = []

for ns in node_list:
    subG = deterministic_subgraph(G,ns)
    subG = trim_edge(subG, keep_top_n = True, keep_span_tree = True, simi_threshold=0.7)
    edge_list.extend(list(subG.edges(data=True)))
    # for u, v, attr in subG.edges():
    #     edge_list.append(subG.edges())

G.clear_edges()
for u, v, attr in edge_list:
    G.add_edge(u, v, **attr)

nx.write_graphml(G, f"{folder_path}/network2.graphml")  

In [ ]:
#%% one-time, generate node image

import shutil
from rdkit import Chem
from tqdm import tqdm

import sys
import os

# current_dir = os.path.dirname(__file__) # for py file
current_dir = os.getcwd() # for ipynb file
dep_dir = os.path.join(current_dir, 'deps')
sys.path.append(dep_dir)

from help_func_mol import get_mol_image

def generate_image(G, attr_name, image_dir = "mol_images" , keep_exist = False):
    '''
    Generate molecule images for nodes in a networkx graph and save them to a directory.
    '''
    if not os.path.exists(image_dir):
        os.makedirs(image_dir)
    else:
        if not keep_exist:
            shutil.rmtree(image_dir)
            os.makedirs(image_dir)

    for node, attr in tqdm(G.nodes(data=True), desc='Generating images', leave=False):

        smiles = attr.get(attr_name, None)
        if smiles is None:
            continue
        mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            continue
        image_path = os.path.join(image_dir, f"node_{node}.png")
        G.nodes[node]['image'] = image_path
        if not os.path.exists(image_path):
            mol_img = get_mol_image(mol)
            mol_img.save(image_path)

folder_path = 'network'
generate_image(G,'smiles', image_dir = f'{folder_path}/mol_images', keep_exist = True)

Draw network

In [ ]:
#%% function - deps - network plot 
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil
import pandas as pd
import networkx as nx
import math

import sys
import os

# current_dir = os.path.dirname(__file__) # for py file
current_dir = os.getcwd() # for ipynb file
dep_dir = os.path.join(current_dir, 'deps')
sys.path.append(dep_dir)

from help_func_network import interpolate_color, plot_graph, trim_edge, compute_grid_layout

In [ ]:
#%% load node list, pos list
import json
import networkx as nx

folder_path = 'network'

with open(f'{folder_path}/community_group1.json', 'r') as f:
    node_list = json.load(f)

with open(f'{folder_path}/node_pos1.json', 'r') as f:
    pos_list = json.load(f)

G = nx.read_graphml(f"{folder_path}/network2.graphml")
image_dir = f'{folder_path}/mol_images'

In [ ]:
#%% color for foodrelated and simi

node_color_palette = {
    'onlyfoodrelate' : 'green',
    'onlysynthetic': 'red',
    'mixinfo' : 'blue',
    'unknown': 'grey',
    '-1': 'grey',
}

for node, data in G.nodes(data=True):
    color_key = data.get('foodrelated', None)
    color = node_color_palette.get('-1', 'grey')
    if node_color_palette.get(color_key, None):
        color = node_color_palette[color_key]
    G.nodes[node]['color'] = color
    image_path = os.path.join(image_dir, f"node_{node}.png")
    G.nodes[node]['image'] = image_path

simi_threshold = 0.7
start_color = "#b3b3b3"      # Color at threshold (lighter)
end_color = "#000000"        # Color at sim=1 (darker)

for i, j, data in G.edges(data=True):
    simi = data['simi']
    fraction = (simi - simi_threshold) / (1 - simi_threshold)
    edge_color = interpolate_color(start_color, end_color, fraction)
    G.edges[i, j]['color'] = edge_color

In [ ]:
#%% calc fig size
import math

clusters = sorted(pos_list, key=lambda c: len(c), reverse=True)

group1 = [c for c in clusters if len(c) > 7] # each large cluster get their own fig
group2 = [c for c in clusters if 3 <= len(c) <= 7] # small cluster all in one grid fig
group3 = [c for c in clusters if len(c) <= 2] # pair and single point in one grid fig

f_sizes = []
for cluster in group1:
    n_count = len(cluster)
    c_size = math.ceil(math.sqrt(n_count))
    # c_size = math.sqrt(n_count)
    f_sizes.append([c_size,c_size])

c_count = len(group2)
sub_f_row = math.ceil(math.sqrt(c_count))
n_count = 7
# c_size = math.ceil(math.sqrt(n_count)
c_size = math.sqrt(n_count)
f_sizes.append([c_size*sub_f_row,c_size*sub_f_row])

n_count = len([node for cluster in group3 for node in cluster])
c_size = math.ceil(math.sqrt(n_count))
f_sizes.append([c_size,c_size])

pix_multi = 1000
f_sizes = [[int(pix_multi*w),int(pix_multi*h)] for w,h in f_sizes]

In [ ]:
#%%  sepe plot for each community
import json
from matplotlib.gridspec import GridSpec
import re

pix_multi = 1000
f_sizes_n = 0
simi_threshold=0.7
dpi = 200

draw_outer_border = True
draw_subplot_inter_border = True

folder_path = 'network/'
os.makedirs(folder_path, exist_ok=True)

# network custom setting
node_image = True
node_halo = True
node_halo_radius = 100

node_label = True
edge_color = True
edge_label = True
image_stroke = True
ax_border = False

for i, pos_list in enumerate(tqdm(group1, leave = False)):
    
    node_list = pos_list.keys()
    f_size_w = f_sizes[f_sizes_n][0]
    f_size_h = f_sizes[f_sizes_n][1]

    subG = G.subgraph(node_list).copy()
    prunedG = trim_edge(subG, simi_threshold=simi_threshold)

    fig, ax = plt.subplots(figsize=(f_size_w/dpi, f_size_h/dpi), dpi=dpi)

    plot_graph(ax, pos_list, prunedG, 
               node_image=node_image, node_label=node_label, 
               node_image_stroke=image_stroke, 
               node_halo = node_halo, node_halo_radius = node_halo_radius,
               edge_color=edge_color, edge_label=edge_label, 
               ax_border=ax_border)
    plt.tight_layout()
    # ax.set_aspect('equal')

    if draw_outer_border:
        global_rect = plt.Rectangle((0, 0), 1, 1,fill=False, edgecolor='black', linewidth=2,transform=fig.transFigure, zorder=5)
        fig.add_artist(global_rect)
        
    fig.savefig(f'{folder_path}/fig_{f_sizes_n:02d}.png', transparent=True, bbox_inches='tight', pad_inches=0)
    plt.close()

    f_sizes_n += 1

if group2:

    f_size_w = f_sizes[f_sizes_n][0]
    f_size_h = f_sizes[f_sizes_n][1]

    n_plots = len(group2)
    n_rows = math.ceil(math.sqrt(n_plots))
    n_cols = math.ceil(n_plots / n_rows)
    fig, axes = plt.subplots(n_rows, n_cols, figsize=(f_size_w/dpi, f_size_h/dpi), dpi=dpi)
    axes = axes.flatten() if n_plots > 1 else [axes]

    for i, pos_list in enumerate(tqdm(group2, leave = False)):
        
        node_list = pos_list.keys()
        node_count = len(node_list)
        size = math.ceil(math.sqrt(node_count))
        subG = G.subgraph(node_list).copy()
        prunedG = trim_edge(subG, simi_threshold=simi_threshold)

        plot_graph(axes[i], pos_list, prunedG, 
               node_image=node_image, node_label=node_label, 
               node_image_stroke=image_stroke,
               node_halo = node_halo, node_halo_radius = node_halo_radius,
               edge_color=edge_color, edge_label=edge_label, 
               )
        
        # axes[i].set_aspect('equal')

    for j in range(i + 1, len(axes)):
        fig.delaxes(axes[j])
    plt.tight_layout()

    if draw_subplot_inter_border:
        for ax in fig.axes:
            ss = ax.get_subplotspec()
            i, j = ss.rowspan.start, ss.colspan.start
            
            left = j / n_cols
            bottom = (n_rows - i - 1) / n_rows
            width = 1 / n_cols
            height = 1 / n_rows
            
            rect = plt.Rectangle((left, bottom), width, height, fill=False, edgecolor='black', linewidth=2, transform=fig.transFigure, zorder=5)
            fig.add_artist(rect)

    if draw_outer_border:
        global_rect = plt.Rectangle((0, 0), 1, 1,fill=False, edgecolor='black', linewidth=2,transform=fig.transFigure, zorder=5)
        fig.add_artist(global_rect)

    fig.savefig(f'{folder_path}fig_{f_sizes_n:02d}.png', transparent=True, bbox_inches='tight', pad_inches=0)
    plt.close()

    f_sizes_n += 1

if group3:

    f_size_w = f_sizes[f_sizes_n][0]
    f_size_h = f_sizes[f_sizes_n][1]

    group3_nodes = [node for cluster in group3 for node in cluster.keys()]

    subG3 = G.subgraph(group3_nodes)
    fig, ax = plt.subplots(figsize=(f_size_w/dpi, f_size_h/dpi), dpi=dpi)

    pos = compute_grid_layout(group3_nodes, dist=len(group3_nodes)/4)
    plot_graph(ax, pos, subG3, 
               node_image=node_image, node_label=node_label, 
               node_image_stroke=image_stroke,
               node_halo = node_halo, node_halo_radius = node_halo_radius,
               edge_color=edge_color, edge_label=edge_label, 
               )
    plt.tight_layout()
    ax.set_aspect('equal')

    if draw_outer_border:
        global_rect = plt.Rectangle((0, 0), 1, 1,fill=False, edgecolor='black', linewidth=2, transform=fig.transFigure, zorder=5)
        fig.add_artist(global_rect)

    fig.savefig(f'{folder_path}fig_{f_sizes_n:02d}.png', transparent=True, bbox_inches='tight', pad_inches=0)
    plt.close()
    f_sizes_n += 1